In [1]:
!pip install discord.py

In [2]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import discord
from discord.ext import commands
import requests

intents = discord.Intents.default()
intents.message_content = True  # Enable message content intent

bot = commands.Bot(command_prefix="!", intents=intents)


WEATHER_API_KEY = "----REPLACE-----"
WEATHER_API_URL = "http://api.openweathermap.org/data/2.5/weather"

TOKEN = "----REPLACE-----"
SERVER_ID = "----REPLACE-----"


def get_weather_data(city, units="metric"):
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": units
    }
    response = requests.get(WEATHER_API_URL, params=params)
    data = response.json()
    return data



@bot.event
async def on_ready():
    print(f"Logged in as {bot.user.name} - {bot.user.id}")
    print("Bot is ready to greet!")

    # Replace with your server's ID
    server = bot.get_guild(SERVER_ID)
    if server:
        print(f"Joined server: {server.name} - {server.id}")

        # Send a greeting message to the server's default channel
        default_channel = server.system_channel
        if default_channel:
            await default_channel.send("Hello everyone! I'm your friendly bot!")


@bot.event
async def on_member_join(member):
    server = member.guild
    default_channel = server.system_channel

    if default_channel:
        await default_channel.send(f"Welcome {member.mention} to the server!")


@bot.command()
async def weather(ctx, *, city):
    data = get_weather_data(city)

    if data.get("cod") == 200:
        weather_main = data["weather"][0]["main"]
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        wind_speed = data["wind"]["speed"]

        weather_embed = discord.Embed(title=f"Weather in {city}", color=discord.Color.blue())
        weather_embed.add_field(name="Main", value=weather_main)
        weather_embed.add_field(name="Description", value=weather_description)
        weather_embed.add_field(name="Temperature", value=f"{temperature} °C")
        weather_embed.add_field(name="Humidity", value=f"{humidity}%")
        weather_embed.add_field(name="Wind Speed", value=f"{wind_speed} m/s")

        await ctx.send(embed=weather_embed)
    else:
        await ctx.send("City not found or an error occurred.")


@bot.command()
async def forecast(ctx, *, city):
    data = get_weather_data(city)

    if data.get("cod") == 200:
        weather_main = data["weather"][0]["main"]
        temperature = data["main"]["temp"]
        forecast_embed = discord.Embed(title=f"Weather Forecast for {city}", color=discord.Color.green())
        forecast_embed.add_field(name="Main", value=weather_main)
        forecast_embed.add_field(name="Temperature", value=f"{temperature} °C")
        forecast_embed.set_footer(text="Note: This is current weather data. Forecasting is not provided.")
        await ctx.send(embed=forecast_embed)
    else:
        await ctx.send("City not found or an error occurred.")


@bot.command()
async def suggest(ctx, *, city):
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}")
    data = response.json()

    if data.get("cod") == "404":
        suggestions = [city["name"] for city in data["cities"]]
        suggestions_text = ", ".join(suggestions)
        await ctx.send(f"Did you mean: {suggestions_text}?")
    else:
        await ctx.send("City found.")


@bot.command(name="bot_help")
async def show_help(ctx):
    help_embed = discord.Embed(title="Bot Commands", color=discord.Color.green())
    help_embed.add_field(name="!forecast <city>", value="Get a weather forecast for a city.")
    help_embed.add_field(name="!suggest <city>", value="Suggest similar city names.")
    help_embed.add_field(name="!get_weather <city>", value="Get the current weather for a city with Descriotion, Wind speed, Temprature, Humidity along with Map link.")  # New field
    await ctx.send(embed=help_embed)


@bot.command(name="get_weather")
async def get_weather(ctx, *, city):
    data = get_weather_data(city)

    if data.get("cod") == 200:
        city_name = data["name"]
        weather_main = data["weather"][0]["main"]
        weather_description = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        wind_speed = data["wind"]["speed"]
        lat = data["coord"]["lat"]
        lon = data["coord"]["lon"]

        weather_embed = discord.Embed(title=f"Weather in {city_name}", color=discord.Color.blue())
        weather_embed.add_field(name="Main", value=weather_main)
        weather_embed.add_field(name="Description", value=weather_description)
        weather_embed.add_field(name="Temperature", value=f"{temperature} °C")
        weather_embed.add_field(name="Humidity", value=f"{humidity}%")
        weather_embed.add_field(name="Wind Speed", value=f"{wind_speed} m/s")
        weather_embed.add_field(name="Location", value=f"Latitude: {lat}, Longitude: {lon}")

        # Add a link to an interactive map based on latitude and longitude
        map_link = f"https://www.openstreetmap.org/?mlat={lat}&mlon={lon}#map=12/{lat}/{lon}"
        weather_embed.add_field(name="Map Link", value=f"[Open Map]({map_link})")

        await ctx.send(embed=weather_embed)
    else:
        await ctx.send("City not found or an error occurred.")


bot.run(TOKEN)


2023-08-20 06:42:23 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-08-20 06:42:23 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 1122d567b1d131757ca94777cb5b58cf).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 1122d567b1d131757ca94777cb5b58cf).


Logged in as WeatherChamp - 1142693515776757830
Bot is ready to greet!
